# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'11-11-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'11-11-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-11-12 05:25:55,33.93911,67.709953,42609,1581,34967,6061.0,Afghanistan,109.454960,3.710484
1,NaN,NaN,NaN,Albania,2020-11-12 05:25:55,41.15330,20.168300,25801,590,12493,12718.0,Albania,896.552922,2.286733
2,NaN,NaN,NaN,Algeria,2020-11-12 05:25:55,28.03390,1.659600,64257,2093,42980,19184.0,Algeria,146.534713,3.257233
3,NaN,NaN,NaN,Andorra,2020-11-12 05:25:55,42.50630,1.521800,5567,75,4488,1004.0,Andorra,7205.073449,1.347225
4,NaN,NaN,NaN,Angola,2020-11-12 05:25:55,-11.20270,17.873900,12953,312,6125,6516.0,Angola,39.411229,2.408708


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,41633,41728,41814,41935,41975,42033,42092,42297,42463,42609
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,21523,21904,22300,22721,23210,23705,24206,24731,25294,25801
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,58574,58979,59527,60169,60800,61381,62051,62693,63446,64257


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1541,1544,1548,1554,1554,1556,1558,1574,1577,1581
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,527,532,536,543,549,557,559,571,579,590
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1980,1980,1999,2011,2024,2036,2048,2062,2077,2093


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,34342,34355,34362,34440,34440,34446,34458,34721,34954,34967
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,11367,11473,11578,11696,11861,12002,12092,12203,12353,12493
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,40577,40577,41001,41244,41510,41783,42037,42325,42626,42980


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2186,2197,2212,2230,2242,2267,2283,2304,2328,2351
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,6985,6995,7061,7097,7134,7188,7226,7263,7348,7409
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1065,1074,1079,1080,1090,1092,1095,1098,1107,1112


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,31,31,31,31,30,30,30,30,30,32
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,71,74,77,77,83,83,83,83,84,84
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,9,9,9,9,9,9,9,9,9,9


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
634,1001.0,Autauga,Alabama,US,2020-11-12 05:25:55,32.539527,-86.644082,2351,32,0,2319.0,"Autauga, Alabama, US",4208.058136,1.361123
671,1075.0,Lamar,Alabama,US,2020-11-12 05:25:55,33.779950,-88.096680,570,7,0,563.0,"Lamar, Alabama, US",4128.938790,1.228070
672,1077.0,Lauderdale,Alabama,US,2020-11-12 05:25:55,34.901719,-87.656247,3088,49,0,3039.0,"Lauderdale, Alabama, US",3330.134047,1.586788


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,10400227,241798,3997175
India,8683916,128121,8066501
Brazil,5747660,163368,5222937


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,10400227,241798,3997175,6161254,2020-11-12 05:25:55,37.936303,-91.379001
India,8683916,128121,8066501,489294,2020-11-12 05:25:55,23.088275,81.806127
Brazil,5747660,163368,5222937,361355,2020-11-12 05:25:55,-12.669522,-48.480493
France,1914918,42599,139177,1733142,2020-11-12 05:25:55,8.314863,-21.695174
Russia,1822345,31326,1360049,430970,2020-11-12 05:25:55,54.546312,62.120860


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
Texas,1022336,19469,0
California,995575,18108,0
Florida,858012,17300,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
Texas,1022336,19469,0,1002867,2020-11-12 05:25:55,31.660643,-98.653069
California,995575,18108,0,977467,2020-11-12 05:25:55,37.843962,-120.728594
Florida,858012,17300,0,840712,2020-11-12 05:25:55,28.940755,-82.700744
New York,540965,33848,0,507117,2020-11-12 05:25:55,42.544151,-75.474183
Illinois,523840,10798,0,513042,2020-11-12 05:25:55,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,327964,7216,0
Illinois,Cook,231462,5881,0
Florida,Miami-Dade,195991,3693,0
Texas,Harris,170835,2877,0
Arizona,Maricopa,170207,3747,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,327964,7216,0,320748,2020-11-12 05:25:55,34.308284,-118.228241,6037.0
Illinois,Cook,231462,5881,0,225581,2020-11-12 05:25:55,41.841448,-87.816588,17031.0
Florida,Miami-Dade,195991,3693,0,192298,2020-11-12 05:25:55,25.611236,-80.551706,12086.0
Texas,Harris,170835,2877,0,167958,2020-11-12 05:25:55,29.858649,-95.393395,48201.0
Arizona,Maricopa,170207,3747,0,166460,2020-11-12 05:25:55,33.348359,-112.491815,4013.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-09,42297,24731,62693,5437,12680,131,1250499,107466,27669,158746,...,3560,69027,0,95149,1215,58838,10,2071,16971,8561
2020-11-10,42463,25294,63446,5477,12816,131,1262476,108687,27671,164866,...,3620,69027,1,95445,1226,59422,10,2071,16997,8610
2020-11-11,42609,25801,64257,5567,12953,131,1273356,110548,27675,172380,...,3700,69397,1,95750,1252,60065,10,2071,17036,8667


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-09,1574,571,2062,75,308,3,33907,1580,907,1454,...,62,588,0,830,35,521,1,605,349,254
2020-11-10,1577,579,2077,75,308,3,34183,1609,907,1499,...,62,588,0,834,35,533,1,605,350,255
2020-11-11,1581,590,2093,75,312,3,34531,1636,907,1564,...,63,591,0,838,35,538,1,605,350,255


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-09,34721,12203,42325,4332,5927,122,1073577,64910,25454,94627,...,2993,66194,0,90054,1087,50877,8,1394,16011,8023
2020-11-10,34954,12353,42626,4405,6036,122,1081897,66835,25454,98663,...,3034,66194,0,90359,1087,51425,8,1394,16019,8040
2020-11-11,34967,12493,42980,4488,6125,124,1081897,68336,25455,103759,...,3078,66672,0,90743,1087,52226,8,1394,16070,8046


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2186,2197,2212,2230,2242,2267,2283,2304,2328,2351
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,535,533,540,541,544,553,554,559,565,570
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,2765,2798,2820,2835,2874,2911,2937,2963,3011,3088
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,946,958,968,977,983,995,1003,1007,1014,1028
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,6736,6749,6776,6812,6851,6878,6903,6926,6952,6994


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-09,204857,20300,0,259699,122811,981297,134537,81432,26908,49,...,287770,998027,134868,2415,1405,193477,118570,28805,286380,18010
2020-11-10,206567,20832,0,263133,124235,989432,138427,82953,27112,49,...,289749,1010364,137385,2462,1413,194912,120011,29316,293812,19242
2020-11-11,208637,21331,0,265163,126197,995575,142402,84741,27342,49,...,293381,1022336,139720,2535,1413,196506,120011,30201,301349,19374


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-11-09        2304    7263    1098  932   2335     664   1054    5217   
2020-11-10        2328    7348    1107  948   2378     665   1061    5254   
2020-11-11        2351    7409    1112  961   2400     668   1062    5282   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-11-09         1489      845  ...     957    209     1047      213   
2020-11-10         1507      857  ...     998    220     1094      222   
2020-11-11         1508      865  ...    1001    226     1099      226   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-11-09            776  1007   568          0      190    310  
2020-11-10            852  1022   614          0      197    321  
2020-11-11            869  1025   627          0      199    323  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-09,3084,84,0,6164,2108,18021,2408,4698,719,0,...,3610,19223,661,59,23,3713,2460,530,2329,114
2020-11-10,3120,92,0,6192,2112,18066,2427,4707,722,0,...,3672,19337,672,59,23,3726,2482,546,2395,127
2020-11-11,3201,96,0,6228,2126,18108,2443,4716,724,0,...,3761,19469,678,59,23,3741,2482,553,2554,127


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-11-09          30      83       9   16     26      17     41      67   
2020-11-10          30      84       9   17     26      18     41      68   
2020-11-11          32      84       9   17     34      19     41      77   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-11-09           48       15  ...       3      4        8        1   
2020-11-10           48       15  ...       3      4        8        1   
2020-11-11           48       15  ...       3      4        8        1   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-11-09              4     2     4          0        7      0  
2020-11-10              4     2     4          0        7      0  
2020-11-11              4     2     4          0        7      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-09,0.004870,0.021689,0.010346,0.010032,0.019866,0.0,0.006695,0.009791,0.000145,0.036519,...,0.013090,0.004321,NaN,0.002803,0.001649,0.011692,0.0,0.000483,0.001003,0.003517
2020-11-10,0.003925,0.022765,0.012011,0.007357,0.010726,0.0,0.009578,0.011362,0.000072,0.038552,...,0.016854,0.000000,inf,0.003111,0.009053,0.009926,0.0,0.000000,0.001532,0.005724
2020-11-11,0.003438,0.020044,0.012783,0.016432,0.010690,0.0,0.008618,0.017123,0.000145,0.045576,...,0.022099,0.005360,0.0,0.003196,0.021207,0.010821,0.0,0.000000,0.002295,0.006620


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-09,0.010270,0.021467,0.006836,0.0,0.003257,0.0,0.01034,0.013470,0.0,0.030475,...,0.000000,0.005128,NaN,0.004843,0.0,0.011650,0.0,0.004983,0.000000,0.003953
2020-11-10,0.001906,0.014011,0.007274,0.0,0.000000,0.0,0.00814,0.018354,0.0,0.030949,...,0.000000,0.000000,NaN,0.004819,0.0,0.023033,0.0,0.000000,0.002865,0.003937
2020-11-11,0.002536,0.018998,0.007703,0.0,0.012987,0.0,0.01018,0.016781,0.0,0.043362,...,0.016129,0.005102,NaN,0.004796,0.0,0.009381,0.0,0.000000,0.000000,0.000000


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-09,0.007632,0.009180,0.006851,0.019774,0.004747,0.000000,0.010035,0.011390,0.000118,0.031706,...,0.016989,0.003380,NaN,0.003331,0.015888,0.009324,0.0,0.013818,0.003824,0.002249
2020-11-10,0.006711,0.012292,0.007112,0.016851,0.018390,0.000000,0.007750,0.029656,0.000000,0.042652,...,0.013699,0.000000,NaN,0.003387,0.000000,0.010771,0.0,0.000000,0.000500,0.002119
2020-11-11,0.000372,0.011333,0.008305,0.018842,0.014745,0.016393,0.000000,0.022458,0.000039,0.051651,...,0.014502,0.007221,NaN,0.004250,0.000000,0.015576,0.0,0.000000,0.003184,0.000746


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-09,0.005744,0.023908,NaN,0.001678,0.007754,0.008825,0.027125,0.042330,0.011465,0.0,...,0.021000,0.007162,0.016943,0.009615,0.000000,0.006775,0.010560,0.014118,0.015856,0.040439
2020-11-10,0.008347,0.026207,NaN,0.013223,0.011595,0.008290,0.028914,0.018678,0.007581,0.0,...,0.006877,0.012361,0.018663,0.019462,0.005694,0.007417,0.012153,0.017740,0.025952,0.068406
2020-11-11,0.010021,0.023954,NaN,0.007715,0.015793,0.006209,0.028715,0.021554,0.008483,0.0,...,0.012535,0.011849,0.016996,0.029651,0.000000,0.008178,0.000000,0.030188,0.025652,0.006860


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-11-09      0.009198  0.005120  0.002740  0.006479  0.016543  0.000000   
2020-11-10      0.010417  0.011703  0.008197  0.017167  0.018415  0.001506   
2020-11-11      0.009880  0.008302  0.004517  0.013713  0.009251  0.004511   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-11-09      0.001901  0.006948  0.002694  0.005952  ...  0.017003   
2020-11-10      0.006641  0.007092  0.012089  0.014201  ...  0.042842   
2020-11-11      0.000943  0.005329  0.000664  0.009335  ...  0.003006   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-11-09     -0.009479  0.031527 -0.009302   0.068871  0.030706  0.030853   
2020-11-10      0.052632  0.044890  0.042254   0.097938  0.014896  0.080986   
2020-11-11      0.027273  0.004570  0.018018   0.019953  0.002935  0.021173   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-11-09            NaN  0.016043  0.013072  
2020-11-10            NaN  0.036842  0.035484  
2020-11-11            NaN  0.010152  0.006231  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-09,0.000000,0.000000,NaN,0.000000,0.011031,0.002671,0.005848,0.005780,0.001393,NaN,...,0.004172,0.002033,0.003035,0.0,0.0,0.001619,0.008610,0.051587,0.007353,0.000000
2020-11-10,0.011673,0.095238,NaN,0.004543,0.001898,0.002497,0.007890,0.001916,0.004172,NaN,...,0.017175,0.005930,0.016641,0.0,0.0,0.003501,0.008943,0.030189,0.028338,0.114035
2020-11-11,0.025962,0.043478,NaN,0.005814,0.006629,0.002325,0.006593,0.001912,0.002770,NaN,...,0.024237,0.006826,0.008929,0.0,0.0,0.004026,0.000000,0.012821,0.066388,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                       \
Admin2           Autauga   Baldwin Barbour    Bibb    Blount   Bullock Butler   
2020-11-09      0.000000  0.000000     0.0  0.0000  0.000000  0.000000    0.0   
2020-11-10      0.000000  0.012048     0.0  0.0625  0.000000  0.058824    0.0   
2020-11-11      0.066667  0.000000     0.0  0.0000  0.307692  0.055556    0.0   

Province_State                              ... Wyoming                  \
Admin2           Calhoun Chambers Cherokee  ...    Park Platte Sheridan   
2020-11-09      0.000000      0.0      0.0  ...     0.0    0.0      0.0   
2020-11-10      0.014925      0.0      0.0  ...     0.0    0.0      0.0   
2020-11-11      0.132353      0.0      0.0  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-11-09          0.0        0.0   0.0   0.0        NaN      0.0    NaN  
2020-11-10          0.0        0.0   0.0   0.0        NaN      0.0    NaN  
2020-11-11          0.0        0.0   0.0   0.0        NaN      0.0    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-09,0.003179,0.021266,0.010330,0.013643,0.015173,0.001225,0.006418,0.015975,0.000207,0.042189,...,0.016631,0.004239,NaN,0.002942,0.001412,0.010267,3.985916e-44,0.000545,0.002276,0.003453
2020-11-10,0.003552,0.022015,0.011170,0.010500,0.012949,0.000613,0.007998,0.013669,0.000140,0.040370,...,0.016743,0.002120,NaN,0.003027,0.005233,0.010096,1.992958e-44,0.000272,0.001904,0.004588
2020-11-11,0.003495,0.021030,0.011976,0.013466,0.011819,0.000306,0.008308,0.015396,0.000142,0.042973,...,0.019421,0.003740,0.0,0.003111,0.013220,0.010459,9.964789e-45,0.000136,0.002099,0.005604


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-09,0.005810,0.014868,0.006405,0.000018,0.007109,1.224928e-63,0.008923,0.015765,1.350396e-07,0.030321,...,0.004349,0.004978,NaN,0.003988,1.347470e-22,0.008448,0.0,0.002605,0.000001,0.005168
2020-11-10,0.003858,0.014439,0.006840,0.000009,0.003555,6.124641e-64,0.008531,0.017060,6.751980e-08,0.030635,...,0.002174,0.002489,NaN,0.004404,6.737348e-23,0.015740,0.0,0.001303,0.001433,0.004553
2020-11-11,0.003197,0.016719,0.007272,0.000005,0.008271,3.062321e-64,0.009356,0.016920,3.375990e-08,0.036999,...,0.009152,0.003796,NaN,0.004600,3.368674e-23,0.012561,0.0,0.000651,0.000717,0.002276


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-09,0.004007,0.009435,0.006556,0.030376,0.018440,0.004392,0.010060,0.015319,0.000178,0.037397,...,0.016081,0.004012,NaN,0.003491,0.008041,0.009648,2.335497e-46,0.006935,0.003450,0.001789
2020-11-10,0.005359,0.010864,0.006834,0.023614,0.018415,0.002196,0.008905,0.022488,0.000089,0.040024,...,0.014890,0.002006,NaN,0.003439,0.004021,0.010209,1.167749e-46,0.003468,0.001975,0.001954
2020-11-11,0.002865,0.011098,0.007569,0.021228,0.016580,0.009295,0.004452,0.022473,0.000064,0.045837,...,0.014696,0.004614,NaN,0.003844,0.002010,0.012893,5.838744e-47,0.001734,0.002579,0.001350


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-09,0.006408,0.025824,NaN,0.004850,0.009407,0.006853,0.026404,0.023101,0.011199,7.706961e-73,...,0.016953,0.007157,0.018670,0.012197,0.002887,0.007441,0.011625,0.019088,0.018381,0.038167
2020-11-10,0.007378,0.026016,NaN,0.009036,0.010501,0.007572,0.027659,0.020889,0.009390,3.853480e-73,...,0.011915,0.009759,0.018666,0.015830,0.004290,0.007429,0.011889,0.018414,0.022166,0.053287
2020-11-11,0.008699,0.024985,NaN,0.008376,0.013147,0.006890,0.028187,0.021222,0.008937,1.926740e-73,...,0.012225,0.010804,0.017831,0.022740,0.002145,0.007804,0.005944,0.024301,0.023909,0.030073


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-11-09      0.008546  0.005508  0.003057  0.006005  0.013124  0.000836   
2020-11-10      0.009482  0.008605  0.005627  0.011586  0.015770  0.001171   
2020-11-11      0.009681  0.008454  0.005072  0.012650  0.012511  0.002841   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-11-09      0.003251  0.007672  0.003907  0.005664  ...  0.028926   
2020-11-10      0.004946  0.007382  0.007998  0.009933  ...  0.035884   
2020-11-11      0.002944  0.006356  0.004331  0.009634  ...  0.019445   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-11-09      0.010388  0.035700  0.004025   0.052837  0.031929  0.021913   
2020-11-10      0.031510  0.040295  0.023139   0.075387  0.023412  0.051449   
2020-11-11      0.029391  0.022433  0.020579   0.047670  0.013174  0.036311   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-11-09           -1.0  0.013715  0.047603  
2020-11-10           -1.0  0.025279  0.041543  
2020-11-11           -1.0  0.017715  0.023887  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-09,0.002355,0.000132,NaN,0.002002,0.009410,0.002142,0.005241,0.003223,0.001615,NaN,...,0.005345,0.003033,0.005911,0.002155,0.001082,0.001824,0.004905,0.031614,0.010350,0.012893
2020-11-10,0.007014,0.047685,NaN,0.003272,0.005654,0.002319,0.006566,0.002570,0.002894,NaN,...,0.011260,0.004482,0.011276,0.001078,0.000541,0.002663,0.006924,0.030901,0.019344,0.063464
2020-11-11,0.016488,0.045582,NaN,0.004543,0.006141,0.002322,0.006579,0.002241,0.002832,NaN,...,0.017749,0.005654,0.010102,0.000539,0.000271,0.003344,0.003462,0.021861,0.042866,0.031732


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                              \
Admin2           Autauga   Baldwin       Barbour      Bibb    Blount   
2020-11-09     -0.002016  0.005848  3.326152e-11  0.008338  0.002500   
2020-11-10     -0.001008  0.008948  1.663076e-11  0.035419  0.001250   
2020-11-11      0.032829  0.004474  8.315380e-12  0.017709  0.154471   

Province_State                                                            ...  \
Admin2               Bullock        Butler   Calhoun  Chambers  Cherokee  ...   
2020-11-09      9.332423e-10  1.525879e-06  0.001469  0.000337  0.000035  ...   
2020-11-10      2.941177e-02  7.629395e-07  0.008197  0.000168  0.000017  ...   
2020-11-11      4.248366e-02  3.814697e-07  0.070275  0.000084  0.000009  ...   

Province_State       Wyoming                                          \
Admin2                  Park    Platte  Sheridan Sublette Sweetwater   
2020-11-09      2.384186e-07  0.009145  0.085872      0.0   0.015625   
2020-11-10      1.192093e-07  0.004573  0.042936      0.0   0.007812   
2020-11-11      5.960464e-08  0.002286  0.021468      0.0   0.003906   

Province_State                                                         
Admin2             Teton     Uinta Unassigned      Washakie    Weston  
2020-11-09      0.007812  0.083335  -0.972376  6.208817e-10 -0.533333  
2020-11-10      0.003906  0.041668  -0.972376  3.104409e-10 -0.533333  
2020-11-11      0.001953  0.020834  -0.972376  1.552204e-10 -0.533333  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201111,AK,21331,NaN,813259,NaN,totalTestsViral,834590,117.0,NaN,...,834590,4,0,54c6a77d7d1f67807c0d8c1ff07f4b3ab0132140,0,0,0,0,0,NaN
1,20201111,AL,208637,32282.0,1245235,NaN,totalTestsViral,1421590,1210.0,21886.0,...,1453872,81,0,a7a21b2a579a0ef9c3c9fdce8e950c09220d5985,0,0,0,0,0,NaN
2,20201111,AR,126197,12254.0,1343051,NaN,totalTestsViral,1456994,801.0,7635.0,...,1469248,14,56,8495663e8c7da850467d0f40ceaab27868d61cd4,0,0,0,0,0,NaN
3,20201111,AS,0,NaN,1768,NaN,totalTestsViral,1768,NaN,NaN,...,1768,0,0,394080f4a2e25af265f7ee8f3e003d764d31c3e2,0,0,0,0,0,NaN
4,20201111,AZ,265163,6926.0,1650072,NaN,totalTestsPeopleViral,1908309,1360.0,22494.0,...,1915235,36,66,d14d6dcfa6e77f38f1296ecd83bb7f92257d2b88,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201111,AK,21331.0,NaN,813259.0,NaN,totalTestsViral,834590.0,117.0,NaN,...,834590,4,0,54c6a77d7d1f67807c0d8c1ff07f4b3ab0132140,0,0,0,0,0,NaN
1,20201111,AL,208637.0,32282.0,1245235.0,NaN,totalTestsViral,1421590.0,1210.0,21886.0,...,1453872,81,0,a7a21b2a579a0ef9c3c9fdce8e950c09220d5985,0,0,0,0,0,NaN
2,20201111,AR,126197.0,12254.0,1343051.0,NaN,totalTestsViral,1456994.0,801.0,7635.0,...,1469248,14,56,8495663e8c7da850467d0f40ceaab27868d61cd4,0,0,0,0,0,NaN
3,20201111,AS,0.0,NaN,1768.0,NaN,totalTestsViral,1768.0,NaN,NaN,...,1768,0,0,394080f4a2e25af265f7ee8f3e003d764d31c3e2,0,0,0,0,0,NaN
4,20201111,AZ,265163.0,6926.0,1650072.0,NaN,totalTestsPeopleViral,1908309.0,1360.0,22494.0,...,1915235,36,66,d14d6dcfa6e77f38f1296ecd83bb7f92257d2b88,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-11-11,AK,21331,NaN,813259,NaN,totalTestsViral,834590,117.0,NaN,NaN,...,834590,4,0,54c6a77d7d1f67807c0d8c1ff07f4b3ab0132140,0,0,0,0,0,NaN
2020-11-11,AL,208637,32282.0,1245235,NaN,totalTestsViral,1421590,1210.0,21886.0,NaN,...,1453872,81,0,a7a21b2a579a0ef9c3c9fdce8e950c09220d5985,0,0,0,0,0,NaN
2020-11-11,AR,126197,12254.0,1343051,NaN,totalTestsViral,1456994,801.0,7635.0,304.0,...,1469248,14,56,8495663e8c7da850467d0f40ceaab27868d61cd4,0,0,0,0,0,NaN
2020-11-11,AS,0,NaN,1768,NaN,totalTestsViral,1768,NaN,NaN,NaN,...,1768,0,0,394080f4a2e25af265f7ee8f3e003d764d31c3e2,0,0,0,0,0,NaN
2020-11-11,AZ,265163,6926.0,1650072,NaN,totalTestsPeopleViral,1908309,1360.0,22494.0,309.0,...,1915235,36,66,d14d6dcfa6e77f38f1296ecd83bb7f92257d2b88,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-11-11,AK,21331.0,NaN,813259.0,NaN,totalTestsViral,834590.0,117.0,NaN,NaN,...,834590,4,0,54c6a77d7d1f67807c0d8c1ff07f4b3ab0132140,0,0,0,0,0,NaN
2020-11-11,AL,208637.0,32282.0,1245235.0,NaN,totalTestsViral,1421590.0,1210.0,21886.0,NaN,...,1453872,81,0,a7a21b2a579a0ef9c3c9fdce8e950c09220d5985,0,0,0,0,0,NaN
2020-11-11,AR,126197.0,12254.0,1343051.0,NaN,totalTestsViral,1456994.0,801.0,7635.0,304.0,...,1469248,14,56,8495663e8c7da850467d0f40ceaab27868d61cd4,0,0,0,0,0,NaN
2020-11-11,AS,0.0,NaN,1768.0,NaN,totalTestsViral,1768.0,NaN,NaN,NaN,...,1768,0,0,394080f4a2e25af265f7ee8f3e003d764d31c3e2,0,0,0,0,0,NaN
2020-11-11,AZ,265163.0,6926.0,1650072.0,NaN,totalTestsPeopleViral,1908309.0,1360.0,22494.0,309.0,...,1915235,36,66,d14d6dcfa6e77f38f1296ecd83bb7f92257d2b88,0,0,0,0,0,NaN
